In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df = pd.read_csv("C:/Users\monster\Desktop/NonLinearRegressionModels\RandomForest\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

"*MODEL TUNING*"

In [4]:
lgb_model = LGBMRegressor()

In [5]:
lgb_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [6]:
lgbm_params = {"learning_rate": [0.1, 0.01, 0.5, 1], "n_estimators": [50, 100, 200, 500, 1000],
               "max_depth": [-1, 1, 5, 10, 20]}

In [8]:
lgbm_cv_model = GridSearchCV(lgb_model, lgbm_params, cv=10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   11.8s finished


In [11]:
lgbm_cv_model.best_params_

{'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 200}

In [91]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.09, max_depth = -1, n_estimators = 200, colsample_bytree = 0.5).fit(X_train, y_train)

In [92]:
y_pred = lgbm_tuned.predict(X_test)

In [93]:
np.sqrt(mean_squared_error(y_test, y_pred))

359.7330282654496